# Wide and Deep on TensorFlow (notebook style)

Copyright 2016 Google Inc. All Rights Reserved. Licensed under the Apache License, Version 2.0 (the "License"); you may not use this file except in compliance with the License. You may obtain a copy of the License at
 http://www.apache.org/licenses/LICENSE-2.0

Unless required by applicable law or agreed to in writing, software distributed under the License is distributed on an "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied. See the License for the specific language governing permissions and limitations under the License.

# Introduction

This notebook uses the `tf.Estimator` API in TensorFlow to answer a yes/no question. This is called a binary classification problem: Given census data about a person such as age, gender, education and occupation (the features), we will try to predict whether or not the person earns more than 50,000 dollars a year (the target label). 

Given an individual's information our model will output a number between 0 and 1, which can be interpreted as the model's certainty that the individual has an annual income of over 50,000 dollars, (1=True, 0=False)


# Imports and constants
First we'll import our libraries and set up some strings for column names. We also print out the version of TensorFlow we are running.

In [ ]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import time

import numpy as np
import tensorflow as tf

tf.logging.set_verbosity(tf.logging.INFO) 
# Set to INFO for tracking training, default is WARN 

print("Using TensorFlow version %s" % (tf.__version__)) 
# This notebook is intended for tested for TF 1.3

CATEGORICAL_COLUMNS = ["workclass", "education", 
                       "marital_status", "occupation", 
                       "relationship", "race", 
                       "gender", "native_country"]

# Columns of the input csv file
COLUMNS = ["age", "workclass", "fnlwgt", "education", 
           "education_num", "marital_status",
           "occupation", "relationship", "race", 
           "gender", "capital_gain", "capital_loss",
           "hours_per_week", "native_country", "income_bracket"]

# Feature columns for input into the model
FEATURE_COLUMNS = ["age", "workclass", "education", 
                   "education_num", "marital_status",
                   "occupation", "relationship", "race", 
                   "gender", "capital_gain", "capital_loss",
                   "hours_per_week", "native_country"]

# Pandas data exploration
We load the data into pandas because it is small enough to manage in memory, and look at some properties.

In [ ]:
import pandas as pd

df = pd.read_csv("adult.test.csv", header=None, names=COLUMNS)

In [ ]:
df.head()

In [ ]:
df.describe(include=[np.number])

In [ ]:
df.describe(include=[np.object])

In [ ]:
df.corr()

# Input file parsing

Here we extract the file into a pandas dataframe and use a built-in utility function to generate an input function for us. TensorFlow also has a similar input function for NumPy arrays.

## More about input functions

The input function is how we will feed the input data into the model during training and evaluation. 
The structure that must be returned is a pair, where the first element is a dict of the column names (features) mapped to a tensor of values, and the 2nd element is a tensor of values representing the answers (labels). Recall that a tensor is just a general term for an n-dimensional array.

This could be represented as: `map(column_name => [Tensor of values]) , [Tensor of labels])`

More concretely, for this particular dataset, something like this:

    { 
      'age':            [ 39, 50, 38, 53, 28, … ], 
      'marital_status': [ 'Married-civ-spouse', 'Never-married', 'Widowed', 'Widowed' … ],
       ...
      'gender':           ['Male', 'Female', 'Male', 'Male', 'Female',, … ], 
    } , 
    [ 1, 1, 0, 1, 1, 1, 0, 0, 0, 0, 1, 1]
    
Additionally, we define which columns of the input data we will treat as categorical vs continuous, using the global `CATEGORICAL_COLUMNS`.

You can try different values for `BATCH_SIZE` to see how they impact your results

In [ ]:
BATCH_SIZE = 40

def generate_input_fn(filename, num_epochs=None, shuffle=True, batch_size=BATCH_SIZE):
    df = pd.read_csv(filename, header=None, names=COLUMNS)
    labels = df["income_bracket"].apply(lambda x: ">50K" in x).astype(int)
    del df["fnlwgt"] # Unused column
    del df["income_bracket"] # Labels column, already saved to labels variable
    
    return tf.estimator.inputs.pandas_input_fn(
        x=df,
        y=labels,
        batch_size=batch_size,
        num_epochs=num_epochs,
        shuffle=shuffle)

print('input function configured')

# Create Feature Columns
This section configures the model with the information about the model. There are many parameters here to experiment with to see how they affect the accuracy.

This is the bulk of the time and energy that is often spent on making a machine learning model work, called *feature selection* or *feature engineering*. We choose the features (columns) we will use for training, and apply any additional transformations to them as needed. 

### Sparse Columns
First we build the sparse columns.

Use `sparse_column_with_keys()` for columns that we know all possible values for.

Use `sparse_column_with_hash_bucket()` for columns that we want the the library to automatically map values for us.

In [ ]:
# The layers module contains many utilities for creating feature columns.

# Categorical base columns.
gender = tf.feature_column.categorical_column_with_vocabulary_list(key="gender", 
                                                                   vocabulary_list=["female", "male"])
race = tf.feature_column.categorical_column_with_vocabulary_list(key="race",
                                                                 vocabulary_list=["Amer-Indian-Eskimo",
                                                                       "Asian-Pac-Islander",
                                                                       "Black", "Other",
                                                                       "White"])

education = tf.feature_column.categorical_column_with_hash_bucket(
  "education", hash_bucket_size=1000)
marital_status = tf.feature_column.categorical_column_with_hash_bucket(
  "marital_status", hash_bucket_size=100)
relationship = tf.feature_column.categorical_column_with_hash_bucket(
  "relationship", hash_bucket_size=100)
workclass = tf.feature_column.categorical_column_with_hash_bucket(
  "workclass", hash_bucket_size=100)
occupation = tf.feature_column.categorical_column_with_hash_bucket(
  "occupation", hash_bucket_size=1000)
native_country = tf.feature_column.categorical_column_with_hash_bucket(
  "native_country", hash_bucket_size=1000)

print('Categorical columns configured')

### Continuous columns
Second, configure the real-valued columns using `real_valued_column()`. 

In [ ]:
# Continuous base columns.
age = tf.feature_column.numeric_column("age")
education_num = tf.feature_column.numeric_column("education_num")
capital_gain = tf.feature_column.numeric_column("capital_gain")
capital_loss  = tf.feature_column.numeric_column("capital_loss")
hours_per_week = tf.feature_column.numeric_column("hours_per_week")

print('Continuous columns configured')

### Transformations
Now for the interesting stuff. We will employ a couple of techniques to get even more out of the data.
 
* **bucketizing** turns what would have otherwise been a continuous feature into a categorical one. 
* **feature crossing** allows us to compute a model weight for specific pairings across columns, rather than learning them as independently. This essentially encodes related columns together, for situations where having 2 (or more) columns being certain values is meaningful. 

Only categorical features can be crossed. This is one reason why age has been bucketized.

For example, crossing education and occupation would enable the model to learn about: 

    education="Bachelors" AND occupation="Exec-managerial"

or perhaps 

    education="Bachelors" AND occupation="Craft-repair"

We do a few combined features (feature crosses) here. 

Add your own, based on your intuitions about the dataset, to try to improve on the model!

In [ ]:
# Transformations.
age_buckets = tf.feature_column.bucketized_column(
    age, boundaries=[ 18, 25, 30, 35, 40, 45, 50, 55, 60, 65 ])

education_occupation = tf.feature_column.crossed_column(
    ["education", "occupation"], hash_bucket_size=int(1e4))

age_race_occupation = tf.feature_column.crossed_column(
    [age_buckets, "race", "occupation"], hash_bucket_size=int(1e6))

country_occupation = tf.feature_column.crossed_column(
    ["native_country", "occupation"], hash_bucket_size=int(1e4))

print('Transformations complete')

### Group feature columns into 2 objects

The wide columns are the sparse, categorical columns that we specified, as well as our hashed, bucket, and feature crossed columns. 

The deep columns are composed of embedded categorical columns along with the continuous real-valued columns. **Column embeddings** transform a sparse, categorical tensor into a low-dimensional and dense real-valued vector. The embedding values are also trained along with the rest of the model. For more information about embeddings, see the TensorFlow tutorial on [Vector Representations Words](https://www.tensorflow.org/tutorials/word2vec/), or [Word Embedding](https://en.wikipedia.org/wiki/Word_embedding) on Wikipedia.

The higher the dimension of the embedding is, the more degrees of freedom the model will have to learn the representations of the features. We are starting with an 8-dimension embedding for simplicity, but later you can come back and increase the dimensionality if you wish.



In [ ]:
# Wide columns and deep columns.
wide_columns = [gender, race, native_country,
      education, occupation, workclass,
      marital_status, relationship,
      age_buckets, education_occupation,
      age_race_occupation, country_occupation]

deep_columns = [
    # Multi-hot indicator columns for columns with fewer possibilities
    tf.feature_column.indicator_column(workclass),
    tf.feature_column.indicator_column(marital_status),
    tf.feature_column.indicator_column(gender),
    tf.feature_column.indicator_column(relationship),
    tf.feature_column.indicator_column(race),
    # Embeddings for categories with more possibilities
    tf.feature_column.embedding_column(education, dimension=8),
    tf.feature_column.embedding_column(native_country, dimension=8),
    tf.feature_column.embedding_column(occupation, dimension=8),
    # Numerical columns
    age,
    education_num,
    capital_gain,
    capital_loss,
    hours_per_week,
]

print('wide and deep columns configured')

# Create the model

You can train either a "wide" model, a "deep" model, or a "wide and deep" model, using the classifiers below. Try each one and see what kind of results you get.

* **Wide**: Linear Classifier
* **Deep**: Deep Neural Net Classifier
* **Wide & Deep**: Combined Linear and Deep Classifier

The `hidden_units` or `dnn_hidden_units` argument is to specify the size of each layer of the deep portion of the network. For example, `[12, 20, 15]` would create a network with the first layer of size 12, the second layer of size 20, and a third layer of size 15.

In [ ]:
def create_model_dir(model_type):
    return 'models/model_' + model_type + '_' + str(int(time.time()))

# If new_model=False, pass in the desired model_dir 
def get_model(model_type, new_model=False, model_dir=None):
    if new_model or model_dir is None:
        model_dir = create_model_dir(model_type) # Comment out this line to continue training a existing model
    print("Model directory = %s" % model_dir)
    
    m = None
    
    # Linear Classifier
    if model_type == 'WIDE':
        m = tf.estimator.LinearClassifier(
            model_dir=model_dir, 
            feature_columns=wide_columns)

    # Deep Neural Net Classifier
    if model_type == 'DEEP':
        m = tf.estimator.DNNClassifier(
            model_dir=model_dir,
            feature_columns=deep_columns,
            hidden_units=[100, 50])

    # Combined Linear and Deep Classifier
    if model_type == 'WIDE_AND_DEEP':
        m = tf.estimator.DNNLinearCombinedClassifier(
                model_dir=model_dir,
                linear_feature_columns=wide_columns,
                dnn_feature_columns=deep_columns,
                dnn_hidden_units=[100, 70, 50, 25])
        
    print('estimator built')
    
    return m, model_dir
    
MODEL_TYPE = 'WIDE_AND_DEEP'
model_dir = create_model_dir(model_type=MODEL_TYPE)
m, model_dir = get_model(model_type = MODEL_TYPE, model_dir=model_dir)

# Fit the model (train it)

Run `train()` to train the model. You can experiment with the `train_steps` and `BATCH_SIZE` parameters.

This can take some time, depending on the values chosen for `train_steps` and `BATCH_SIZE`.


In [ ]:
%%time 

train_file = str("adult.data.csv") 
# "gs://cloudml-public/census/data/adult.data.csv"
# storage.googleapis.com/cloudml-public/census/data/adult.data.csv

m.train(input_fn=generate_input_fn(train_file), 
      steps=1000)

print('training done')

# Evaluate the accuracy of the model
Let's see how the model did. We will evaluate all the test data.

In [ ]:
test_file  = str("adult.test.csv") 
# "gs://cloudml-public/census/data/adult.test.csv"
# storage.googleapis.com/cloudml-public/census/data/adult.test.csv

results = m.evaluate(input_fn=generate_input_fn(test_file, num_epochs=1, shuffle=False), 
                     steps=None)
print('evaluate done')
print('\nAccuracy: %s' % results['accuracy'])

# Make a prediction

In [ ]:
# Create a dataframe to wrap in an input function
df = pd.read_csv("adult.test.csv", header=None, names=COLUMNS)
data_predict = df.iloc[8000:8005]
data_predict.head() # show this before deleting, so we know what the labels are

In [ ]:
# If you run this cell twice, it will give an error since you'd be deleting something that was already gone
del data_predict["fnlwgt"] # Unused column
del data_predict["income_bracket"] # remove the label column

In [ ]:
predict_input_fn = tf.estimator.inputs.pandas_input_fn(
        x=data_predict,
        batch_size=1,
        num_epochs=1,
        shuffle=False)

predictions = m.predict(input_fn=predict_input_fn)

for prediction in predictions:
    print("Predictions:    {} with probabilities {}\n".format(prediction["classes"], prediction["probabilities"]))

# Export a model optimized for inference
We can upload our trained model to the Cloud Machine Learning Engine's Prediction Service, which will take care of serving our model and scaling it. The code below exports our trained model to a `saved_model.pb` file and a `variables` folder where the trained weights are stored. This format is also compatible with TensorFlow Serving.

The `export_savedmodel()` function expects a `serving_input_receiver_fn()`, which returns the mapping from the data that the Prediction Service passes in to the data that should be fed into the trained TensorFlow prediction graph.

In [ ]:
def column_to_dtype(column):
    if column in CATEGORICAL_COLUMNS:
        return tf.string
    else:
        return tf.float32
    
feature_spec = {
    column: tf.FixedLenFeature(shape=[1], dtype=column_to_dtype(column))
        for column in FEATURE_COLUMNS
}
serving_fn = tf.estimator.export.build_parsing_serving_input_receiver_fn(feature_spec)
m.export_savedmodel(export_dir_base=model_dir + '/export', 
                            serving_input_receiver_fn=serving_fn)

# Conclusions

In this Juypter notebook, we have configured, created, and evaluated a Wide & Deep machine learning model, that combines the powers of a Linear Classifier with a Deep Neural Network, using TensorFlow's tf.Estimator module.

With this working example in your toolbelt, you are ready to explore the wide (and deep) world of machine learning with TensorFlow! Some ideas to help you get going:
* Change the features we used today. Which columns do you think are correlated and should be crossed? Which ones do you think are just adding noise and could be removed to clean up the model?
* Swap in an entirely new dataset! There are many dataset available on the web, or use a dataset you possess! Check out https://archive.ics.uci.edu/ml to find your own dataset. 